# ML Hackathon Project — Hangman Solver using HMM + Q-Learning

## Overview
This project implements an intelligent Hangman solver using a combination of **Hidden Markov Models (HMM)** and **Reinforcement Learning (Q-Learning)**.  
It learns the statistical structure of English words and adapts its guessing strategy over time.

## Features
- Enhanced HMM with unigram, bigram, and trigram frequency modeling.
- Q-learning agent that optimizes letter-guessing decisions.
- End-to-end training and testing pipeline with modular components.
- Supports model persistence and evaluation across rounds.

## Usage
Place your `corpus.txt` and `test.txt` files in the same directory as this notebook, then run all cells in order.  
The system will train the HMM, initialize the Q-agent, and begin solving Hangman rounds automatically.



In [ ]:
import numpy as np
from collections import defaultdict, Counter
from tqdm import tqdm
import pickle
import random

# ============================================================================
# HMM
# ============================================================================

class ImprovedHMM:
    """Enhanced HMM with better word matching and frequency analysis"""

    def __init__(self):
        self.words_by_length = defaultdict(list)
        self.letter_freq = Counter()
        self.position_freq = defaultdict(lambda: defaultdict(Counter))
        self.bigram_freq = defaultdict(Counter)
        self.trigram_freq = defaultdict(lambda: defaultdict(Counter))
        self.first_letter_freq = Counter()
        self.last_letter_freq = Counter()
        self.pattern_cache = {}

    def train(self, words):
        print("Training Enhanced HMM...")

        for word in words:
            word = word.lower().strip()
            if not word or not word.isalpha():
                continue

            length = len(word)
            self.words_by_length[length].append(word)

            # Global frequency
            self.letter_freq.update(word)

            # Positional frequency
            for i, char in enumerate(word):
                self.position_freq[length][i][char] += 1

            # Bigrams
            for i in range(len(word) - 1):
                self.bigram_freq[word[i]][word[i+1]] += 1

            # Trigrams
            for i in range(len(word) - 2):
                self.trigram_freq[word[i]][word[i+1]][word[i+2]] += 1

            # First and last letters
            if len(word) > 0:
                self.first_letter_freq[word[0]] += 1
                self.last_letter_freq[word[-1]] += 1

        # Normalize frequencies
        total = sum(self.letter_freq.values())
        self.letter_freq = {k: v/total for k, v in self.letter_freq.items()}

        print(f"Trained on {sum(len(v) for v in self.words_by_length.values())} words")
        print(f"Top 10 letters: {sorted(self.letter_freq.items(), key=lambda x: -x[1])[:10]}")

    def get_matches(self, mask, guessed):
        """Find matching words with caching"""
        cache_key = (mask, tuple(sorted(guessed)))
        if cache_key in self.pattern_cache:
            return self.pattern_cache[cache_key]

        length = len(mask)
        if length not in self.words_by_length:
            return []

        matches = []
        for word in self.words_by_length[length]:
            if self._matches(word, mask, guessed):
                matches.append(word)

        # Cache result (limit cache size)
        if len(self.pattern_cache) < 10000:
            self.pattern_cache[cache_key] = matches

        return matches

    def _matches(self, word, mask, guessed):
        """Check if word matches mask and constraints"""
        for w, m in zip(word, mask):
            if m != '_':
                if w != m:
                    return False
            else:
                if w in guessed:
                    return False
        return True

    def get_letter_probabilities(self, mask, guessed):
        """Get probability distribution over letters"""
        remaining = set('abcdefghijklmnopqrstuvwxyz') - guessed

        if not remaining:
            return {}

        combined_scores = Counter()

        # Strategy 1: Direct pattern matching (HIGHEST WEIGHT)
        matches = self.get_matches(mask, guessed)

        if 0 < len(matches) <= 2000:
            for word in matches:
                word_letters = set(word) & remaining
                for letter in word_letters:
                    count = word.count(letter)
                    combined_scores[letter] += count * 1000

        # Strategy 2: Positional frequency
        length = len(mask)
        if length in self.position_freq:
            for i, m in enumerate(mask):
                if m == '_' and i in self.position_freq[length]:
                    for letter in remaining:
                        combined_scores[letter] += self.position_freq[length][i].get(letter, 0) * 100

        # Strategy 3: Trigram context
        for i, m in enumerate(mask):
            if m == '_':
                # Left bigram context
                if i >= 2 and mask[i-2] != '_' and mask[i-1] != '_':
                    left2, left1 = mask[i-2], mask[i-1]
                    if left2 in self.trigram_freq and left1 in self.trigram_freq[left2]:
                        for letter in remaining:
                            combined_scores[letter] += self.trigram_freq[left2][left1].get(letter, 0) * 200

                # Right bigram context
                if i <= len(mask) - 3 and mask[i+1] != '_' and mask[i+2] != '_':
                    right1, right2 = mask[i+1], mask[i+2]
                    for letter in remaining:
                        if letter in self.trigram_freq and right1 in self.trigram_freq[letter]:
                            combined_scores[letter] += self.trigram_freq[letter][right1].get(right2, 0) * 200

        # Strategy 4: Bigram context
        for i, m in enumerate(mask):
            if m == '_':
                if i > 0 and mask[i-1] != '_':
                    left = mask[i-1]
                    for letter in remaining:
                        combined_scores[letter] += self.bigram_freq[left].get(letter, 0) * 150

                if i < len(mask) - 1 and mask[i+1] != '_':
                    right = mask[i+1]
                    for letter in remaining:
                        combined_scores[letter] += self.bigram_freq[letter].get(right, 0) * 150

        # Strategy 5: First/last letter
        blank_positions = [i for i, m in enumerate(mask) if m == '_']
        if 0 in blank_positions:
            for letter in remaining:
                combined_scores[letter] += self.first_letter_freq.get(letter, 0) * 80

        if len(mask) - 1 in blank_positions:
            for letter in remaining:
                combined_scores[letter] += self.last_letter_freq.get(letter, 0) * 80

        # Strategy 6: Global frequency (base score)
        for letter in remaining:
            combined_scores[letter] += self.letter_freq.get(letter, 0) * 500

        # Vowel boost in early game
        revealed_count = sum(1 for c in mask if c != '_')
        vowels = set('aeiou') & remaining
        if revealed_count < len(mask) * 0.3 and len(guessed) < 5:
            for v in vowels:
                combined_scores[v] += 800

        # Normalize to probabilities
        total = sum(combined_scores.values())
        if total > 0:
            probabilities = {k: v/total for k, v in combined_scores.items()}
        else:
            # Uniform distribution fallback
            probabilities = {k: 1.0/len(remaining) for k in remaining}

        return probabilities




In [ ]:
# ============================================================================
# HANGMAN ENVIRONMENT
# ============================================================================

class HangmanEnv:
    """Hangman game environment for RL"""

    def __init__(self, words, max_lives=6):
        self.words = words
        self.max_lives = max_lives
        self.reset()

    def reset(self, word=None):
        """Reset environment with a new word"""
        if word is None:
            self.word = random.choice(self.words).lower().strip()
        else:
            self.word = word.lower().strip()

        self.mask = ['_'] * len(self.word)
        self.guessed = set()
        self.lives = self.max_lives
        self.done = False
        self.won = False

        return self._get_state()

    def step(self, letter):
        """Take action (guess letter) and return (state, reward, done)"""
        if self.done:
            return self._get_state(), 0, True, {}

        letter = letter.lower()

        # Penalize repeated guesses
        if letter in self.guessed:
            reward = -5
            return self._get_state(), reward, self.done, {'repeated': True}

        self.guessed.add(letter)

        # Check if letter in word
        if letter in self.word:
            # Correct guess
            num_occurrences = 0
            for i, c in enumerate(self.word):
                if c == letter:
                    self.mask[i] = c
                    num_occurrences += 1

            # Reward based on number of letters revealed
            reward = 5 * num_occurrences

            # Check if won
            if '_' not in self.mask:
                self.done = True
                self.won = True
                reward += 50  # Big bonus for winning
        else:
            # Wrong guess
            self.lives -= 1
            reward = -10

            # Check if lost
            if self.lives <= 0:
                self.done = True
                self.won = False
                reward = -50  # Big penalty for losing

        return self._get_state(), reward, self.done, {}

    def _get_state(self):
        """Get current state representation"""
        return {
            'mask': ''.join(self.mask),
            'guessed': set(self.guessed),
            'lives': self.lives,
            'revealed_count': sum(1 for c in self.mask if c != '_'),
            'length': len(self.word)
        }



In [ ]:
# ============================================================================
# Q-LEARNING RL AGENT
# ============================================================================

class QLearningAgent:
    """Q-Learning agent that uses HMM probabilities"""

    def __init__(self, hmm, alpha=0.1, gamma=0.95, epsilon=0.3):
        self.hmm = hmm
        self.Q = defaultdict(lambda: defaultdict(float))
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.05

    def get_state_key(self, state):
        """Convert state to hashable key"""
        mask = state['mask']
        guessed_str = ''.join(sorted(state['guessed']))
        lives = state['lives']
        return f"{mask}|{guessed_str}|{lives}"

    def choose_action(self, state, training=True):
        """Choose action using epsilon-greedy with HMM guidance"""
        remaining = set('abcdefghijklmnopqrstuvwxyz') - state['guessed']

        if not remaining:
            return None

        # Exploration
        if training and random.random() < self.epsilon:
            # Explore with HMM guidance (weighted random)
            hmm_probs = self.hmm.get_letter_probabilities(state['mask'], state['guessed'])
            if hmm_probs:
                letters = list(hmm_probs.keys())
                probs = np.array([hmm_probs[l] for l in letters])
                probs = probs / probs.sum()
                return np.random.choice(letters, p=probs)
            else:
                return random.choice(list(remaining))

        # Exploitation: choose best action based on Q-values + HMM
        state_key = self.get_state_key(state)
        hmm_probs = self.hmm.get_letter_probabilities(state['mask'], state['guessed'])

        best_score = float('-inf')
        best_action = None

        for letter in remaining:
            # Combine Q-value with HMM probability
            q_value = self.Q[state_key][letter]
            hmm_prob = hmm_probs.get(letter, 0)

            # Weighted combination (Q-learning gets more weight as training progresses)
            combined_score = 0.6 * q_value + 0.4 * hmm_prob * 100

            if combined_score > best_score:
                best_score = combined_score
                best_action = letter

        return best_action if best_action else random.choice(list(remaining))

    def update(self, state, action, reward, next_state, done):
        """Update Q-value"""
        state_key = self.get_state_key(state)
        next_state_key = self.get_state_key(next_state)

        # Current Q-value
        current_q = self.Q[state_key][action]

        # Best next Q-value
        if done:
            max_next_q = 0
        else:
            remaining_next = set('abcdefghijklmnopqrstuvwxyz') - next_state['guessed']
            if remaining_next:
                max_next_q = max([self.Q[next_state_key][a] for a in remaining_next])
            else:
                max_next_q = 0

        # Q-learning update
        new_q = current_q + self.alpha * (reward + self.gamma * max_next_q - current_q)
        self.Q[state_key][action] = new_q

    def decay_epsilon(self):
        """Decay exploration rate"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


In [ ]:
# ============================================================================
# TRAINING
# ============================================================================

def train_rl_agent(hmm, train_words, num_episodes=10000, max_lives=6):
    """Train RL agent"""
    print(f"\nTraining RL Agent for {num_episodes} episodes...")

    env = HangmanEnv(train_words, max_lives)
    agent = QLearningAgent(hmm)

    rewards_history = []
    success_history = []

    for episode in tqdm(range(num_episodes)):
        state = env.reset()
        episode_reward = 0

        while not env.done:
            action = agent.choose_action(state, training=True)

            if action is None:
                break

            next_state, reward, done, info = env.step(action)

            agent.update(state, action, reward, next_state, done)

            episode_reward += reward
            state = next_state

        rewards_history.append(episode_reward)
        success_history.append(1 if env.won else 0)

        # Decay exploration
        if episode % 100 == 0:
            agent.decay_epsilon()

        # Progress report
        if (episode + 1) % 500 == 0:
            recent_success = np.mean(success_history[-500:])
            recent_reward = np.mean(rewards_history[-500:])
            print(f"Episode {episode+1}: Success Rate: {recent_success:.2%}, Avg Reward: {recent_reward:.1f}, Epsilon: {agent.epsilon:.3f}")

    return agent, rewards_history, success_history


In [ ]:
# ============================================================================
# EVALUATION
# ============================================================================

def evaluate_agent(agent, test_words, max_lives=6, verbose=True):
    """Evaluate trained agent on test set"""
    if verbose:
        print(f"\nEvaluating on {len(test_words)} words...")

    env = HangmanEnv(test_words, max_lives)

    successes = 0
    total_wrong = 0
    total_repeated = 0

    iterator = tqdm(test_words) if verbose else test_words

    for word in iterator:
        state = env.reset(word)

        while not env.done:
            action = agent.choose_action(state, training=False)

            if action is None:
                break

            next_state, reward, done, info = env.step(action)

            if 'repeated' in info:
                total_repeated += 1
            elif reward < 0:  # Wrong guess (not repeated)
                total_wrong += 1

            state = next_state

        if env.won:
            successes += 1

    # Calculate metrics
    n = len(test_words)
    success_rate = successes / n
    final_score = (success_rate * n) - (total_wrong * 5) - (total_repeated * 2)

    if verbose:
        print("\n" + "="*60)
        print("EVALUATION RESULTS")
        print("="*60)
        print(f"Total Games: {n}")
        print(f"Wins: {successes}")
        print(f"Success Rate: {success_rate:.4f} ({success_rate*100:.2f}%)")
        print(f"Total Wrong Guesses: {total_wrong}")
        print(f"Average Wrong per Game: {total_wrong/n:.2f}")
        print(f"Total Repeated Guesses: {total_repeated}")
        print(f"Average Repeated per Game: {total_repeated/n:.4f}")
        print("-"*60)
        print(f"FINAL SCORE = ({success_rate:.4f} * {n}) - ({total_wrong} * 5) - ({total_repeated} * 2)")
        print(f"FINAL SCORE = {success_rate * n:.2f} - {total_wrong * 5:.2f} - {total_repeated * 2:.2f}")
        print(f"FINAL SCORE = {final_score:.2f}")
        print("="*60)

    return success_rate, total_wrong, total_repeated, final_score


In [ ]:
if __name__ == "__main__":
    # Load data
    print("Loading corpus...")
    with open("corpus.txt") as f:
        corpus = [line.strip() for line in f if line.strip()]

    print("Loading test set...")
    with open("test.txt") as f:
        test_words = [line.strip() for line in f if line.strip()]

    # Train HMM
    hmm = ImprovedHMM()
    hmm.train(corpus)

    # --------------------------------------------------------------------------
    # QUICK EVALUATION OF HMM-ONLY ACCURACY (WITHOUT RL)
    # --------------------------------------------------------------------------
    print("\nEvaluating pure HMM accuracy (no RL)...")
    correct = 0
    total = 500  # limit to 500 random test words for quick evaluation

    for word in random.sample(test_words, min(total, len(test_words))):
        mask = "_" * len(word)
        guessed = set()
        lives = 6

        while "_" in mask and lives > 0:
            probs = hmm.get_letter_probabilities(mask, guessed)
            if not probs:
                break
            guess = max(probs, key=probs.get)
            guessed.add(guess)
            if guess in word:
                mask = "".join([c if c == guess or mask[i] != "_" else "_" for i, c in enumerate(word)])
            else:
                lives -= 1

        if "_" not in mask:
            correct += 1

    print(f"HMM-only Success Rate: {correct}/{total} = {correct/total:.2%}")



Loading corpus...
Loading test set...
Training Enhanced HMM...
Trained on 49979 words
Top 10 letters: [('e', 0.10365422049194833), ('a', 0.08866740821262145), ('i', 0.08857892829756217), ('o', 0.07543544758197032), ('r', 0.07073547876060705), ('n', 0.0701814259591644), ('t', 0.06781564156365076), ('s', 0.06118596792813746), ('l', 0.05773525124082548), ('c', 0.04575254274422563)]

Evaluating pure HMM accuracy (no RL)...
HMM-only Success Rate: 161/500 = 32.20%


In [ ]:

    # Train RL Agent
    # Use subset of corpus for faster training
    train_subset = corpus[:10000] if len(corpus) > 10000 else corpus
    agent, rewards, successes = train_rl_agent(hmm, train_subset, num_episodes=10000)

    # Evaluate on test set
    success_rate, total_wrong, total_repeated, final_score = evaluate_agent(agent, test_words)


    # Plot learning curves
    print("\nTraining complete!")
    print(f"Final Success Rate: {success_rate:.2%}")
    print(f"Final Score: {final_score:.2f}")


Training RL Agent for 10000 episodes...


  5%|▌         | 505/10000 [00:17<06:52, 23.01it/s]

Episode 500: Success Rate: 22.60%, Avg Reward: -38.3, Epsilon: 0.293


 10%|█         | 1003/10000 [00:33<04:14, 35.35it/s]

Episode 1000: Success Rate: 26.60%, Avg Reward: -32.0, Epsilon: 0.285


...

Episode 9500: Success Rate: 29.20%, Avg Reward: -28.8, Epsilon: 0.186


100%|██████████| 10000/10000 [05:19<00:00, 31.33it/s]


Episode 10000: Success Rate: 24.60%, Avg Reward: -34.4, Epsilon: 0.182

Evaluating on 2000 words...


100%|██████████| 2000/2000 [00:52<00:00, 37.89it/s]


EVALUATION RESULTS
Total Games: 2000
Wins: 645
Success Rate: 0.3225 (32.25%)
Total Wrong Guesses: 10420
Average Wrong per Game: 5.21
Total Repeated Guesses: 0
Average Repeated per Game: 0.0000
------------------------------------------------------------
FINAL SCORE = (0.3225 * 2000) - (10420 * 5) - (0 * 2)
FINAL SCORE = 645.00 - 52100.00 - 0.00
FINAL SCORE = -51455.00

Training complete!
Final Success Rate: 32.25%
Final Score: -51455.00


In [ ]:
# ============================================================================
# DEMO: Watch the trained agent play Hangman interactively
# ============================================================================

def demo_game(agent, hmm, word=None, max_lives=6):
    """Show one game being played step-by-step"""
    env = HangmanEnv([word] if word else corpus, max_lives)
    state = env.reset(word)
    print("\n🎮 Starting a new Hangman game!")
    print(f"Word length: {len(env.word)} letters\n")

    step = 0
    while not env.done:
        step += 1
        mask_display = " ".join(env.mask)
        print(f"Step {step:02d} | Mask: {mask_display} | Lives: {env.lives} | Guessed: {' '.join(sorted(env.guessed))}")

        # Agent chooses next action
        action = agent.choose_action(state, training=False)
        next_state, reward, done, info = env.step(action)
        print(f"→ Agent guesses: '{action.upper()}' | Reward: {reward:+}")
        print("-" * 50)

        state = next_state

    if env.won:
        print(f"✅ The agent WON! The word was: {env.word.upper()} 🎉")
    else:
        print(f"❌ The agent LOST! The word was: {env.word.upper()}")
    print("=" * 60)


# ---------------------------------------------------------------------------
# Run a few demo games
# ---------------------------------------------------------------------------
print("\n\n========== DEMO GAMES ==========")
for i in range(3):
    demo_word = random.choice(test_words)
    demo_game(agent, hmm, word=demo_word)




========== DEMO GAMES ==========

🎮 Starting a new Hangman game!
Word length: 7 letters

Step 01 | Mask: _ _ _ _ _ _ _ | Lives: 6 | Guessed: 
→ Agent guesses: 'E' | Reward: -10
--------------------------------------------------
Step 02 | Mask: _ _ _ _ _ _ _ | Lives: 5 | Guessed: e
→ Agent guesses: 'A' | Reward: +5
--------------------------------------------------
Step 03 | Mask: _ _ _ a _ _ _ | Lives: 5 | Guessed: a e
→ Agent guesses: 'T' | Reward: -10
--------------------------------------------------
Step 04 | Mask: _ _ _ a _ _ _ | Lives: 4 | Guessed: a e t
→ Agent guesses: 'R' | Reward: -10
--------------------------------------------------
Step 05 | Mask: _ _ _ a _ _ _ | Lives: 3 | Guessed: a e r t
→ Agent guesses: 'L' | Reward: -10
--------------------------------------------------
Step 06 | Mask: _ _ _ a _ _ _ | Lives: 2 | Guessed: a e l r t
→ Agent guesses: 'N' | Reward: +5
--------------------------------------------------
Step 07 | Mask: _ _ _ a n _ _ | Lives: 2 | Guessed: 